# Data wrangling

## Qué se hizo 
1) Se definieron utilidades de evidencia (resúmenes del target, límites IQR, colapso de categorías raras y guardado robusto Parquet/CSV).  
2) Se implementó `wrangle_posts_for_score()` que:
   - Toma un snapshot **antes** (shape, nulos, dtypes, stats del score y top de flairs).
   - Limpia y enriquece: elimina columnas con fuga (`post_id`, `url`, `permalink`, `thumbnail`), imputa `link_flair_text`, deriva calendarios (`year`, `month`, `dayofweek`, `hour`, `is_weekend`, `recency_days`), calcula longitudes (`title_len`, `selftext_len`), normaliza categóricas y colapsa niveles raros.
   - Trata outliers: crea `score_clipped` con IQR y `num_comments_capped` al p99.
   - Tipifica columnas, elimina constantes y selecciona un set ordenado de features + `score` y `score_clipped`.
   - Toma un snapshot **después** (shape, nulos, dtypes, stats del target y top flairs) y, si se pide, guarda a disco.
   - Imprime evidencias comparables: tabla before/after, nulos, conteo de dtypes, cardinalidades, top de flairs y **un histograma comparativo `score` vs `score_clipped`** (se anuncia con el `print`: *"[FIGURA] Histograma comparativo: score (antes) vs score_clipped (después) — reducción de dispersión extrema"*).
3) Se ejecutó un bloque de **evidencias post-wrangling**:
   - Comparación del target crudo vs clipped en tabla side-by-side.
   - **Matriz de correlaciones** numéricas y ranking de correlaciones con el target.
   - Selección de features, **split** train/test reproducible sin sklearn, guardado de CSVs y verificación de la comparabilidad del target por split mediante histograma y tabla de estadísticas.

---

## Evidencias clave observadas
- **Estado del dataset (antes→después):**  
  - Filas/columnas: 1542×16 → 1542×17.  
  - Se removieron fugas: `post_id`, `url`, `permalink`, `thumbnail`.  
  - Se eliminaron constantes: `locked`, `year`, `is_weekend`.  
  - Cardinalidad: `author` se colapsa 1351→7; `link_flair_text` 122→48; `subreddit` se mantiene en 27.  

- **Top `link_flair_text` (antes→después):**  
  - `None`→`unknown` (368), normalización a minúsculas y agrupación de raras en `other`.  

- **Reducción de outliers (clipping del target):**  
  - `score_before` → **mean 935.04**, **std 4038.68**, **p50 22**, **p95 4436.65**, **max 65830.00**.  
  - `score_after_clipped` → **mean 120.36**, **std 170.02**, **p50 22**, **p95 450.50**, **max 450.50**.  
  - *Insight:* el clipping reduce drásticamente escala y dispersión manteniendo la mediana, estabilizando el objetivo para modelado.

- **Correlaciones con el target (`score_clipped`):**  
  - Positivas: `num_comments_capped` **0.491**, `recency_days` **0.417**, `dayofweek` **0.364**, `title_len` **0.230**.  
  - Negativas: `month` **−0.416**, `is_self` **−0.160**, `selftext_len` **−0.090**.  
  - *Insight:* el engagement (comentarios) y patrones temporales pesan más que la longitud del contenido; los posts de enlace tienden a rendir mejor que los de texto (`is_self` negativo).

- **Matriz de correlaciones (post-wrangling):**  
  - No se observan multicolinealidades fuertes entre numéricas; la estructura es mayormente débil salvo las relaciones anteriores.

- **Features seleccionadas (10):**  
  `['num_comments_capped', 'recency_days', 'dayofweek', 'title_len', 'selftext_len', 'is_self', 'month', 'link_flair_text', 'subreddit', 'author']`.

- **Split y comparabilidad del target (`score_clipped`):**  
  - Tamaños: **Train (1233, 11)** | **Test (309, 11)**.  
  - **Train:** mean 124.68, std 172.61, p50 24, p95 450.50, max 450.50.  
  - **Test:** mean 103.13, std 158.34, p50 17, p95 450.50, max 450.50.  
  - El histograma por split confirma distribuciones alineadas; p95 y max coinciden.

- **Persistencia:**  
  Datasets procesados guardados en:  
  `../data/processed/train_posts_clean.csv`  
  `../data/processed/test_posts_clean.csv`.

---

## Decisiones derivadas
- Usar **`score_clipped`** como objetivo operativo (o evaluar `log1p(score)` si se modela el crudo).  
- Priorizar señales de **interacción y tiempo** (`num_comments_capped`, `recency_days`, calendarios) y mantener `is_self`, `title_len`; codificar/gestionar cardinalidad para `link_flair_text`, `subreddit`, `author`.  
- Mantener excluidas las columnas con fuga y considerar validación temporal si se amplía el horizonte de predicción.


In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

RAW_DIR = Path("../data/raw")
RAW_DIR.mkdir(parents=True, exist_ok=True)

# Cargar desde SQLite
conn = sqlite3.connect("../db/imedia.sqlite")
comments = pd.read_sql("SELECT * FROM comments;", conn)
conn.close()

# Guardar copia inmutable
comments.to_parquet(RAW_DIR / "comments_raw.parquet", index=False)

comments.head()


,comment_id,post_id,author,body,created_utc,parent_id,link_id,score,is_submitter
0,ng4q0es,1nq1588,Present_Tonight1813,I made a program that prompts the user for a s...,1.758810e+09,t3_1nq1588,t3_1nq1588,2,0
1,ng4cg2y,1nq1588,cptsdemon,I made a tool called [PyLiveDev](https://pypi....,1.758805e+09,t3_1nq1588,t3_1nq1588,2,0
2,ng6dq2g,1nq1588,Fr1dge21,As my first project I managed to automate stoc...,1.758827e+09,t3_1nq1588,t3_1nq1588,4,0
3,ng8r3e5,1nq1588,AdventPriest,"Full disclosure, I've leaned heavily on AI to ...",1.758856e+09,t1_ng4aj2a,t3_1nq1588,1,0
4,ng5ys9z,1nq1588,geovane_jeff,My own backup app :D saves me every week!,1.758822e+09,t3_1nq1588,t3_1nq1588,4,0


In [4]:
import sqlite3

db_path = "../db/imedia.sqlite"
conn = sqlite3.connect(db_path)

# Mostrar las tablas disponibles
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tablas disponibles:")
print(tables)

# Leer los datos de cada tabla
authors = pd.read_sql_query("SELECT * FROM authors;", conn)
comments = pd.read_sql_query("SELECT * FROM comments;", conn)
posts = pd.read_sql_query("SELECT * FROM posts;", conn)
subreddits = pd.read_sql_query("SELECT * FROM subreddits;", conn)

Tablas disponibles:
         name
0  subreddits
1     authors
2       posts
3    comments


In [2]:
INTERIM_DIR = Path("../data/interim")
INTERIM_DIR.mkdir(parents=True, exist_ok=True)

df = comments.copy()

# Flags y columnas útiles
df['has_author'] = df['author'].notna()
df['created_dt'] = pd.to_datetime(df['created_utc'], unit='s')

# Eliminar textos vacíos o removidos
mask_valid = ~df['body'].isin(["[deleted]", "[removed]"])
df = df[mask_valid].copy()

# Normalizar encoding del texto
df['body'] = df['body'].astype(str).str.normalize('NFKC')

df.to_parquet(INTERIM_DIR / "comments_clean_step1.parquet", index=False)
df.head()


,comment_id,post_id,author,body,created_utc,parent_id,link_id,score,is_submitter,has_author,created_dt
0,ng4q0es,1nq1588,Present_Tonight1813,I made a program that prompts the user for a s...,1.758810e+09,t3_1nq1588,t3_1nq1588,2,0,True,2025-09-25 14:15:12
1,ng4cg2y,1nq1588,cptsdemon,I made a tool called [PyLiveDev](https://pypi....,1.758805e+09,t3_1nq1588,t3_1nq1588,2,0,True,2025-09-25 13:04:46
2,ng6dq2g,1nq1588,Fr1dge21,As my first project I managed to automate stoc...,1.758827e+09,t3_1nq1588,t3_1nq1588,4,0,True,2025-09-25 18:59:23
3,ng8r3e5,1nq1588,AdventPriest,"Full disclosure, I've leaned heavily on AI to ...",1.758856e+09,t1_ng4aj2a,t3_1nq1588,1,0,True,2025-09-26 02:58:48
4,ng5ys9z,1nq1588,geovane_jeff,My own backup app :D saves me every week!,1.758822e+09,t3_1nq1588,t3_1nq1588,4,0,True,2025-09-25 17:46:52


In [3]:
# Medidas textuales
df['text_len'] = df['body'].str.len()
df['word_count'] = df['body'].str.split().str.len()

# Señales lingüísticas
df['has_exclamation'] = df['body'].str.contains('!', regex=False)
df['has_question'] = df['body'].str.contains('?', regex=False)

# Quitar comentarios con menos de 3 caracteres (ruido)
df = df[df['text_len'] > 2].copy()

df.to_parquet(INTERIM_DIR / "comments_features_step2.parquet", index=False)
df[['text_len','word_count','has_exclamation','has_question']].describe()


,text_len,word_count
count,4681.000000,4681.000000
mean,157.043367,27.638752
std,228.275117,39.535044
min,3.000000,1.000000
25%,42.000000,7.000000
50%,93.000000,16.000000
75%,185.000000,33.000000
max,5576.000000,859.000000


In [4]:
# Winsorization ligera en score
score_q_low, score_q_high = df['score'].quantile([0.01, 0.99])
df['score_clipped'] = df['score'].clip(score_q_low, score_q_high)

# Winsorization en longitud del texto
len_q_high = df['text_len'].quantile(0.99)
df['text_len_clipped'] = df['text_len'].clip(upper=len_q_high)

df.to_parquet(INTERIM_DIR / "comments_outliers_handled.parquet", index=False)

df[['score','score_clipped','text_len','text_len_clipped']].describe()


,score,score_clipped,text_len,text_len_clipped
count,4681.000000,4681.000000,4681.000000,4681.000000
mean,57.244606,39.426148,157.043367,151.445289
std,402.919899,143.206810,228.275117,177.530690
min,-81.000000,-1.000000,3.000000,3.000000
25%,1.000000,1.000000,42.000000,42.000000
50%,4.000000,4.000000,93.000000,93.000000
75%,16.000000,16.000000,185.000000,185.000000
max,14829.000000,1136.400000,5576.000000,1058.200000


In [5]:
import re

def clean_text(t):
    t = t.lower()
    t = re.sub(r"http\S+", "", t)           # URLs
    t = re.sub(r"[^a-zA-Z0-9?!.,'’ ]", " ", t) 
    t = re.sub(r"\s+", " ", t).strip()
    return t

df['clean_text'] = df['body'].apply(clean_text)

# Quitar textos extremadamente cortos tras limpieza
df = df[df['clean_text'].str.len() > 3].copy()

df.to_parquet(INTERIM_DIR / "comments_text_normalized.parquet", index=False)

df[['body','clean_text']].head()


,body,clean_text
0,I made a program that prompts the user for a s...,i made a program that prompts the user for a s...
1,I made a tool called [PyLiveDev](https://pypi....,i made a tool called pylivedev specifically to...
2,As my first project I managed to automate stoc...,as my first project i managed to automate stoc...
3,"Full disclosure, I've leaned heavily on AI to ...","full disclosure, i've leaned heavily on ai to ..."
4,My own backup app :D saves me every week!,my own backup app d saves me every week!


In [12]:
from sklearn.model_selection import train_test_split

PROCESSED_DIR = Path("../data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Selección de columnas finales para modelado
model_df = df[[
    "comment_id",
    "clean_text",
    "score_clipped",
    "text_len_clipped",
    "word_count",
    "has_exclamation",
    "has_question",
    "created_dt",
    "is_submitter",
]].copy()

# Asegurar que el texto limpio exista
model_df = model_df.dropna(subset=["clean_text"])

# Etiqueta binaria (sentimiento proxy basado en score_clipped)
median_score = model_df["score_clipped"].median()
model_df["sentiment"] = (model_df["score_clipped"] >= median_score).astype(int)

# Split estratificado: 70% train, 15% val, 15% test
train_df, temp_df = train_test_split(
    model_df,
    test_size=0.30,
    random_state=42,
    stratify=model_df["sentiment"],
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,
    random_state=42,
    stratify=temp_df["sentiment"],
)

# Guardar datasets procesados
train_df.to_parquet(PROCESSED_DIR / "sentiment_train.parquet", index=False)
val_df.to_parquet(PROCESSED_DIR / "sentiment_val.parquet", index=False)
test_df.to_parquet(PROCESSED_DIR / "sentiment_test.parquet", index=False)

train_df.head()


,comment_id,clean_text,score_clipped,text_len_clipped,word_count,has_exclamation,has_question,created_dt,is_submitter,sentiment
3369,noaemkf,same thing with my cat except her name was toa...,40.0,202.0,18,False,True,2025-11-11 14:52:36,0,1
495,no5c89x,yes. i refuse to believe alexander skarsg rd n...,35.0,67.0,12,False,False,2025-11-10 18:28:44,0,1
4486,nqr42tj,"nta, your sister went full blown asshole by po...",42.0,162.0,29,False,False,2025-11-25 19:14:24,0,1
1460,noatq80,then trade it for an old twinkie.,1.0,33.0,7,False,False,2025-11-11 16:08:58,0,0
3284,no7qgr2,omfg we need to see her back rear loaf now!!,1.0,44.0,10,True,False,2025-11-11 02:18:38,0,0


In [1]:
# ===============================
# CHUNK: Verificación de splits procesados
# ===============================

from pathlib import Path
import pandas as pd

PROCESSED_DIR = Path("../data/processed")

TRAIN_PATH = PROCESSED_DIR / "sentiment_train.parquet"
VAL_PATH   = PROCESSED_DIR / "sentiment_val.parquet"
TEST_PATH  = PROCESSED_DIR / "sentiment_test.parquet"

# Cargar splits
train_chk = pd.read_parquet(TRAIN_PATH)
val_chk   = pd.read_parquet(VAL_PATH)
test_chk  = pd.read_parquet(TEST_PATH)

# Función auxiliar para imprimir info útil de cada split
def inspect_df(df, name: str):
    print(f"\n==========================")
    print(f"📌 {name} — shape={df.shape}")
    print("==========================\n")

    print("▶ Head:")
    display(df.head())

    print("\n▶ Info:")
    display(df.info())

    print("\n▶ Null counts:")
    display(df.isna().sum())

    print("\n▶ Describe numérico:")
    display(df.describe())

    print("\n▶ Distribución de etiqueta 'sentiment':")
    if 'sentiment' in df.columns:
        display(df['sentiment'].value_counts(normalize=True))
    else:
        print("No existe columna 'sentiment' en este split.")

# Inspeccionar los 3 splits
inspect_df(train_chk, "TRAIN")
inspect_df(val_chk, "VAL")
inspect_df(test_chk, "TEST")



📌 TRAIN — shape=(3245, 10)

▶ Head:


,comment_id,clean_text,score_clipped,text_len_clipped,word_count,has_exclamation,has_question,created_dt,is_submitter,sentiment
0,noaemkf,same thing with my cat except her name was toa...,40.0,202.0,18,False,True,2025-11-11 14:52:36,0,1
1,no5c89x,yes. i refuse to believe alexander skarsg rd n...,35.0,67.0,12,False,False,2025-11-10 18:28:44,0,1
2,nqr42tj,"nta, your sister went full blown asshole by po...",42.0,162.0,29,False,False,2025-11-25 19:14:24,0,1
3,noatq80,then trade it for an old twinkie.,1.0,33.0,7,False,False,2025-11-11 16:08:58,0,0
4,no7qgr2,omfg we need to see her back rear loaf now!!,1.0,44.0,10,True,False,2025-11-11 02:18:38,0,0



▶ Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3245 entries, 0 to 3244
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   comment_id        3245 non-null   object        
 1   clean_text        3245 non-null   object        
 2   score_clipped     3245 non-null   float64       
 3   text_len_clipped  3245 non-null   float64       
 4   word_count        3245 non-null   int64         
 5   has_exclamation   3245 non-null   bool          
 6   has_question      3245 non-null   bool          
 7   created_dt        3245 non-null   datetime64[ns]
 8   is_submitter      3245 non-null   int64         
 9   sentiment         3245 non-null   int64         
dtypes: bool(2), datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 209.3+ KB


None


▶ Null counts:


...

comment_id          0
clean_text          0
score_clipped       0
text_len_clipped    0
word_count          0
has_exclamation     0
has_question        0
created_dt          0
is_submitter        0
sentiment           0
dtype: int64


▶ Describe numérico:


,score_clipped,text_len_clipped,word_count,created_dt,is_submitter,sentiment
count,696.000000,696.000000,696.000000,696,696.000000,696.000000
mean,38.969828,150.278448,27.757184,2025-11-14 02:14:19.579022848,0.008621,0.540230
min,-1.000000,4.000000,1.000000,2025-09-25 11:58:42,0.000000,0.000000
25%,1.000000,42.000000,7.000000,2025-11-11 00:23:54,0.000000,0.000000
50%,4.000000,89.000000,15.000000,2025-11-11 05:52:53.500000,0.000000,1.000000
75%,15.250000,178.000000,33.000000,2025-11-11 16:39:31.750000128,0.000000,1.000000
max,1136.400000,1058.200000,497.000000,2025-12-02 06:16:01,1.000000,1.000000
std,131.737275,181.174129,39.806112,NaN,0.092513,0.498737



▶ Distribución de etiqueta 'sentiment':


sentiment
1    0.54023
0    0.45977
Name: proportion, dtype: float64